In [87]:
import pandas as pd
import statsmodels.api as sm

In [100]:

cluster1_phase2 = pd.read_csv('data_stationnaires/Cluster1_Phase_II.csv')
cluster1_phase3 = pd.read_csv('data_stationnaires/Cluster1_Phase_III.csv')
cluster0_phase2 = pd.read_csv('data_stationnaires/Cluster0_Phase_II.csv')
cluster0_phase3 = pd.read_csv('data_stationnaires/Cluster0_Phase_III.csv')

cluster1_phase2.drop(columns=['Date','Unnamed: 0'], inplace=True)
cluster1_phase3.drop(columns=['Date','Unnamed: 0'], inplace=True)
cluster0_phase2.drop(columns=['Date','Unnamed: 0'], inplace=True)
cluster0_phase3.drop(columns=['Date','Unnamed: 0'], inplace=True)

cluster1_phase2.dropna(inplace=True)
cluster1_phase3.dropna(inplace=True)
cluster0_phase2.dropna(inplace=True)
cluster0_phase3.dropna(inplace=True)




Cluster 1 Phase 2

In [101]:
y_1 = cluster1_phase2['Price']
X_1 = cluster1_phase2.drop(columns=['Price'])

Test 1: STEPWISE REGRESSION



critère R2 - AIC - BIC - p-values: balance entre complexité et performance AIC et BIC -> performance et complexité, R2 performance

In [102]:

y_1 = cluster1_phase2['Price']
X_1 = cluster1_phase2.drop(columns=['Price'])

X_1 = sm.add_constant(X_1)
p_value_threshold = 0.05
# Initial full model with all variables
model = sm.OLS(y_1, X_1).fit()
print(model.summary())

while True:
    current_aic = model.aic
    current_bic = model.bic
    current_adj_r_squared = model.rsquared_adj
    if max(model.pvalues) > p_value_threshold:
        # Remove the least significant variable
        worst_feature = model.pvalues.idxmax()
        X_temp = X_1.drop(columns=[worst_feature])
        model_temp = sm.OLS(y_1, X_temp).fit()
        
        # Check if removing worsens AIC, BIC, or adjusted R^2
        if model_temp.aic < current_aic and model_temp.bic < current_bic and model_temp.rsquared_adj > current_adj_r_squared:
         
            X_1 = X_temp
            model = model_temp
        else:
            break  
    else:
        break
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.866
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     1.234
Date:                Thu, 27 Feb 2025   Prob (F-statistic):              0.391
Time:                        18:36:53   Log-Likelihood:                 122.97
No. Observations:                  57   AIC:                            -149.9
Df Residuals:                       9   BIC:                            -51.87
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -3.5298      2

Refaire la meme chose mais avec HAC pour gérer l'hétéroscédasticité

In [103]:
import numpy as np

y_1 = cluster1_phase2['Price']
X_1 = cluster1_phase2.drop(columns=['Price'])

y_1 = cluster1_phase2['Price']
T = len(y_1)
maxlags = int(4 * (T / 100)**(2/9)) 
model = sm.OLS(y_1, X_1).fit()
robust_se_model = model.get_robustcov_results(cov_type='HAC', maxlags=maxlags)

print(robust_se_model.summary())

while True:
    current_aic = robust_se_model.aic
    current_bic = robust_se_model.bic
    current_adj_r_squared = robust_se_model.rsquared_adj
    if max(robust_se_model.pvalues) > p_value_threshold:
        # Remove the least significant variable
        worst_feature = int(np.argmax(robust_se_model.pvalues))
        worst_feature = X_1.columns[worst_feature]
        X_temp = X_1.drop(columns=[worst_feature])
        model_temp = sm.OLS(y_1, X_temp).fit()
        model_temp = model_temp.get_robustcov_results(cov_type='HAC', maxlags=maxlags)
        
        # Check if removing worsens AIC, BIC, or adjusted R^2
        if model_temp.aic < current_aic and model_temp.bic < current_bic and model_temp.rsquared_adj > current_adj_r_squared:
            X_1 = X_temp
            robust_se_model = model_temp
        else:
            break  
    else:
        break
print(robust_se_model.summary())

                                 OLS Regression Results                                
Dep. Variable:                  Price   R-squared (uncentered):                   0.829
Model:                            OLS   Adj. R-squared (uncentered):              0.024
Method:                 Least Squares   F-statistic:                              961.8
Date:                Thu, 27 Feb 2025   Prob (F-statistic):                    4.70e-14
Time:                        18:36:53   Log-Likelihood:                          115.25
No. Observations:                  57   AIC:                                     -136.5
Df Residuals:                      10   BIC:                                     -40.48
Df Model:                          47                                                  
Covariance Type:                  HAC                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

Newey-West HAC

In [104]:
T = len(y_1)
maxlags = int(4 * (T / 100)**(2/9))
maxlags

3

Le code que vous avez mentionné illustre l'utilisation de la méthode Ordinary Least Squares (OLS) pour ajuster un modèle linéaire, mais avec une extension spécifique: l'ajustement des erreurs standard pour prendre en compte la possibilité d'autocorrélation et d'hétéroscédasticité dans les résidus, en utilisant les erreurs standard robustes de Newey-West.

LASSO

In [110]:
from sklearn.linear_model import LassoCV

y_1 = cluster1_phase2['Price']
X_1 = cluster1_phase2.drop(columns=['Price'])

lasso_cv = LassoCV(cv=15, random_state=0)  # cv est le nombre de plis dans la validation croisée

# Ajuster le modèle aux données
lasso_cv.fit(X_1, y_1)

# Alpha optimal trouvé par CV
alpha_optimal = lasso_cv.alpha_
print("Alpha optimal:", alpha_optimal)

# Coefficients du modèle avec le meilleur alpha
coefficients = dict(zip(X_1.columns, lasso_cv.coef_))
for k,v in coefficients.items():
    if v!=0:
        print(k,":",v)

# Intercept avec le meilleur alpha
intercept = lasso_cv.intercept_
print("Intercept:", intercept)

c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.2557065730221564e-05, tolerance: 2.7884609537569557e-05
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.285981255984118e-05, tolerance: 2.7884609537569557e-05
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the numbe

Alpha optimal: 0.03045037527473941
InVolReg : -0.0002078905251559886
InVolFin : -0.0005169804829401383
OutVolFin : 0.0007213375042292796
InVolFin_lag2 : -0.0010710086318111067
TVolFin_lag1 : 0.0037547359330835322
Intercept: -0.041542278251593956


c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0003270673781329003, tolerance: 3.063201540061189e-05
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.001244118805196398, tolerance: 3.063201540061189e-05
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\zoero\OneDrive\Bureau\M2\S1\projet tutoré\code projet tuteuré\Banking-EU-ETS\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of